<a href="https://colab.research.google.com/github/dqrtec/IA/blob/master/formiga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import random as rdn
import threading

In [72]:
class formiga:
    caminho = None
    matrizRainhas = None
    matrizFeromonio = None
    rainhas = 0
    custo = 0
    
    def __init__(self, quantidadeRainhas):
        self.rainhas = quantidadeRainhas
        self.custo = quantidadeRainhas
        self.caminho = self.caminhoInicial()
        
        
    
    def caminhoInicial(self):
        a = [i for i in range(self.rainhas)] 
        rdn.shuffle(a)
        return a
    
    def geraMatrizRainhas(self):
        self.matrizRainhas = [[0 for i in range(self.rainhas)] for j in range(self.rainhas)]

        for l, c in enumerate(self.caminho):
            self.matrizRainhas[l][c] = 1
    
    def calcularCusto(self):
        dicionario = {}
        for i in self.caminho:
            if i in dicionario:
                dicionario[i] = dicionario[i] + 1
            else:
                dicionario[i] = 1
            
        self.custo = int(self.custoDiagonal() / 2) + self.custoColuna()
    
    def custoDiagonal(self):
        somaDiagonal = 0 
        
        for i in range(len(self.caminho)):
            for j in range(i+1, len(self.caminho)):

                if self.caminho[i]+(j-i) < len(self.caminho):
                    if self.caminho[i+(j-i)] == self.caminho[i]+(j-i):
                        somaDiagonal += 1
                if self.caminho[i]-(j-i) >= 0:
                    if self.caminho[i+(j-i)] == self.caminho[i]-(j-i):
                        somaDiagonal += 1

            for j in range(0,i):
                if self.caminho[i]+(j+1) < len(self.caminho):
                    if self.caminho[i-(j+1)] == self.caminho[i]+(j+1):
                        somaDiagonal += 1
                if self.caminho[i]-(j+1) >= 0:
                    if self.caminho[i-(j+1)] == self.caminho[i]-(j+1):
                        somaDiagonal += 1
        
        return somaDiagonal
    
    def custoColuna(self):
        custo = 0
        for i in range(len(self.caminho)):
            for j in range(i+1, len(self.caminho)):
                if self.caminho[i] == self.caminho[j]:
                    custo += 1
        return custo
    
    def criarMatrizFeromonio(self):
        self.matrizFeromonio = [[1 for i in range(self.rainhas)] for j in range(self.rainhas)]
    
    def calcularMatrizFeromonio(self):
        self.calcularCusto()
        if (self.custo == 0):
            return self.caminho

        feromonio = round(1 / self.custo, 3)

        for l, c in enumerate(self.caminho):
            self.matrizFeromonio[l][c] += feromonio
                
                
    def calcMatrizProb(self):
        matrizProb = []
        somas = []
        
        for line in self.matrizFeromonio:
            somas.append(0)
            
        for line in self.matrizFeromonio:    
            for index, col in enumerate(line):
                somas[index] += col

        for indexLine, line in enumerate(self.matrizFeromonio):
            matrizProb.append([])
            
            for indexCol, col in enumerate(line):
                matrizProb[indexLine].append(round(col / somas[indexCol], 3))
                      
        return matrizProb
        
    def novoCaminhoFormiga(self, matrizProb):
        novoCaminho = [0 for n in matrizProb]
        i = 0
        for line in matrizProb:
            roleta = [0 for n in line]
            pesos = [n for n in line]
            somaPeso = sum(pesos)
            sorteio = rdn.random()
            
            prob = [x*100/somaPeso for x in line]
            
            roleta[0] = (0, prob[0])
            for x in range(1, len(prob)):
                roleta[x] = (roleta[x-1][1], roleta[x-1][1] + prob[x])

            novoCaminho[i] = self.escolherPosicaoArbitraria(roleta, sorteio)
            i += 1

        self.caminho = novoCaminho
    
    def escolherPosicaoArbitraria(self, roleta, sorteio):
        sorteio *= 100
        posicao = 0
        for minimo, maximo in roleta:
            
            if sorteio >= minimo and sorteio < maximo:
                return posicao
            
            posicao += 1
        
        return posicao
    
    def debug(self):             
        print('Rainhas')
        self.printMatriz(self.matrizRainhas)
                
    def printMatriz(self, matriz):
        for linha in matriz:
            for coluna in linha:
                print(coluna, end='  ')
            print()
        print()
    
    def encontrarSolucao(self):
        while (self.custo > 0):
            self.geraMatrizRainhas()
            self.criarMatrizFeromonio()
            self.calcularMatrizFeromonio()

            matrizProb = self.calcMatrizProb()
            self.novoCaminhoFormiga(matrizProb)
            
        if self.custo == 0:
            return self

In [83]:
from multiprocessing.pool import ThreadPool

In [84]:
def main(rainhas, quantidadeFormigas):
    formigas = []
    
    for novaFormiga in range(quantidadeFormigas):
        formigas.append(formiga(rainhas))
        
    for novaFormiga in formigas:
        pool = ThreadPool(processes=1)
        async_result = pool.apply_async(novaFormiga.encontrarSolucao, None)
#         threadFormiga = threading.Thread(target=novaFormiga.encontrarSolucao, args=("thread sendo executada",))
#         threadFormiga.start()
        novaFormiga.encontrarSolucao()
    
        if async_result.get().custo == 0:
            return novaFormiga

In [ ]:
def foo(bar, baz):
    return 'foo' + baz



async_result = pool.apply_async(foo, ('world', 'foo')) # tuple of args for foo

# do some other stuff in the main process

return_val = async_result.get() 

In [85]:
formigaSolucao = main(6, 1)
formigaSolucao.debug()

TypeError: encontrarSolucao() argument after * must be an iterable, not NoneType